In [1]:
import psycopg2
import arcpy
import arcgis
import pandas as pd
import numpy as np
import datetime
import requests
#import seaborn as sns
import matplotlib.pyplot as plt
#import rasterio
import pyodbc
import os
import warnings
import subprocess
import sys
#from shapely.geometry import Point
#import pyproj

# sample 1/200 of the dataset

In [13]:
# Read cell size in X direction
cellSizeX = arcpy.Describe(r"E:/coursework/ARCGIS2/LAB3/LAB3/dem.tif").meanCellWidth * 200

# Read cell size in Y direction
cellSizeY = arcpy.Describe(r"E:/coursework/ARCGIS2/LAB3/LAB3/dem.tif").meanCellHeight * 200

# Resample
cellSizeXY = u"{} {}".format(cellSizeX, cellSizeY)
cellSizeXY
#arcpy.Resample_management("c:/data/image.tif", "resample.tif", cellSizeXY, "NEAREST")

'0.06714694282904643 0.06714694282904635'

In [14]:
arcpy.management.Resample(r"E:/coursework/ARCGIS2/LAB3/LAB3/dem.tif", r"E:/coursework/ARCGIS2/LAB3/LAB3/resampled_dem.tif", cellSizeXY, "NEAREST")

<Result 'E:\\coursework\\ARCGIS2\\LAB3\\LAB3\\resampled_dem.tif'>

# Convert raster into point shapefile

In [15]:
arcpy.conversion.RasterToPoint("E:/coursework/ARCGIS2/LAB3/LAB3/resampled_dem.tif", r"E:/coursework/ARCGIS2/LAB3/LAB3/resampled_dem_point.shp", "Value")

<Result 'E:\\coursework\\ARCGIS2\\LAB3\\LAB3\\resampled_dem_point.shp'>

# IDW interpolation

In [16]:
#out_raster = arcpy.ga.Idw("E:/coursework/ARCGIS2/LAB3/LAB3/resampled_dem_point.shp", "grid_code", 500, 2, "VARIABLE 12", None); out_raster.save(r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb\Idw_RasterT_2")

#IDW interporlation
arcpy.env.workspace = "E:/coursework/ARCGIS2/LAB3/LAB3"
input_fc = "resampled_dem_point.shp"
geostat_layer =  "dem_idw_geo.lyr"
output_raster = "dem_idw.tif"

# Set up the IDW interpolation parameters
cell_size = 0.023 # Cell size of the output raster
power = 2 # Power parameter for IDW interpolation (optional, default is 2)

arcpy.CheckOutExtension("Spatial")
arcpy.ga.IDW(input_fc, 'grid_code',geostat_layer, output_raster,cell_size = cell_size, power = power)
#arcpy.sa.Idw(input_fc, "max_tmpf", output_raster, cell_size, power)

<Result 'dem_idw_geo.lyr'>

# Kriging 

In [17]:
#Bayesian Kriging
geostat_layer =  "dem_BayesianKriging_geo.lyr"
output_raster = "dem_BayesianKriging.tif"
arcpy.ga.EmpiricalBayesianKriging(input_fc, 'grid_code', geostat_layer, output_raster, cell_size = cell_size)
#arcpy.sa.Idw(input_fc, "max_tmpf", output_raster, cell_size, power)

<Result 'dem_BayesianKriging_geo.lyr'>

# RBF


In [18]:
#Radial Basis Functions
geostat_layer =  "dem_RBF_geo.lyr"
output_raster = "dem_RBF.tif"
arcpy.ga.RadialBasisFunctions(input_fc, 'grid_code', geostat_layer, output_raster, cell_size = cell_size)


<Result 'dem_RBF_geo.lyr'>

# Interpolation assessment


In [19]:
arcpy.CrossValidation_ga("dem_idw_geo.lyr","dem_idw_CV.shp")
arcpy.CrossValidation_ga("dem_BayesianKriging_geo.lyr","dem_BayesianKriging_CV.shp")
arcpy.CrossValidation_ga("dem_RBF_geo.lyr","dem_RBF_CV.shp")

<Result 'E:/coursework/ARCGIS2/LAB3/LAB3\\dem_RBF_CV.shp'>

In [20]:
# Convert attribute table to a NumPy array and pd DataFrame
arcpy.env.workspace = "E:/coursework/ARCGIS2/LAB3/LAB3"

idw_cv_array = arcpy.da.FeatureClassToNumPyArray("dem_idw_cv.shp", ["FID", "Error"])
idw_cv_pd = pd.DataFrame(idw_cv_array, columns=["FID", "Error"])

Kriging_cv_array = arcpy.da.FeatureClassToNumPyArray("dem_BayesianKriging_cv.shp", ["FID", "Error"])
Kriging_cv_pd = pd.DataFrame(Kriging_cv_array, columns=["FID", "Error"])

RBF_cv_array = arcpy.da.FeatureClassToNumPyArray("dem_RBF_cv.shp", ["FID", "Error"])
RBF_cv_pd = pd.DataFrame(RBF_cv_array, columns=["FID", "Error"])

# Calculate squared error
idw_cv_pd['squared_error'] = idw_cv_pd['Error'] ** 2
Kriging_cv_pd['squared_error'] = Kriging_cv_pd['Error'] ** 2
RBF_cv_pd['squared_error'] = RBF_cv_pd['Error'] ** 2

# Calculate RMSE
idw_rmse = numpy.sqrt(idw_cv_pd['squared_error'].mean())
Kriging_rmse = numpy.sqrt(Kriging_cv_pd['squared_error'].mean())
RBF_rmse = numpy.sqrt(RBF_cv_pd['squared_error'].mean())

#Print and compare results
print(f'The RMSE for IDW is: {idw_rmse}')
print(f'The RMSE for Kriging is: {Kriging_rmse}')
print(f'The RMSE for RBF is: {RBF_rmse}')


The RMSE for IDW is: 57.22564964560843
The RMSE for Kriging is: 48.2514619432444
The RMSE for RBF is: 52.677574897310826


In [21]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'Lab3',
                             user = 'postgres',
                             password = 'zyx11457')

# Set the name of the new table in the PostgreSQL database
output_table = "dem_assesment_table"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"CREATE TABLE {output_table} (method text, rmse double precision)")

# Use psycopg2 to copy the data from the Geodatabase table to the PostgreSQL table
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('IDW',idw_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('Kriging',Kriging_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('RBF',RBF_rmse))


# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

# Upload Interporlation Map (Kriging) to POSTGIS

In [22]:
#convert raster to point
arcpy.conversion.RasterToPoint(r"dem_BayesianKriging.tif",'dem_BayesianKriging_point.shp','VALUE')

#reproject
#outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 15N')
#arcpy.management.Project("temperature_BayesianKriging_point.shp", r"temperature_BayesianKriging_point_reproject.shp", outCS)

#clip
arcpy.analysis.Clip("dem_BayesianKriging_point.shp", r"shp_bdry_state/Boundaries_of_Minnesota.shp", r"dem_BayesianKriging_point_clip.shp", None)

<Result 'E:/coursework/ARCGIS2/LAB3/LAB3\\dem_BayesianKriging_point_clip.shp'>

# upload using shp2pgsql

In [25]:
#upload using shp2pgsql
# Set the necessary parameters
shp2pgsql_path = r"D:\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"D:\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/dem_BayesianKriging_point_clip.shp"
srid = 4326
schema_table = "dem_BayesianKriging"
host = "35.193.80.209"
port = 5432
user = "postgres"
db_name = "Lab3"
password = "zyx11457"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")


Return code: 0
Error message: could not print result table: Invalid argument
Field pointid is an FTDouble with width 10 and precision 0
Field grid_code is an FTDouble with width 13 and precision 11
Shapefile type: Point
Postgis type: POINT[2]



# Download geojson from url using flask

In [26]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.125.39.137:5000/dem"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"E:/coursework/ARCGIS2/LAB3/LAB3"
    output_file_path = os.path.join(output_folder, "dem_inteporlation.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: E:/coursework/ARCGIS2/LAB3/LAB3\dem_inteporlation.geojson
GeoJSON file saved as E:/coursework/ARCGIS2/LAB3/LAB3\dem_inteporlation.geojson


# Convert geojson into esri geojson

In [27]:
import json

# Read input JSON data from file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type":"FeatureCollection",
    "features":[
        {
            "type":"Feature",
            "geometry":{
                "type":"Point",
                "coordinates":point_data["geometry"]["coordinates"],
            },
            "properties":{"elevation": point_data["grid_code"]},
        } for point_data in input_data
    ] 
}

# Write output JSON data to file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation_esri.geojson', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)


# upload feature class to arconline

In [28]:
import arcgis
gis = arcgis.gis.GIS("home")

# File previously loaded into AGOL (manually)
file_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation_esri.geojson"

# Add item 
props = {"title":"dem_inteporlation", "description":"Lab 3", "tags":"GIS 5572", "type":"GeoJson"}
item = gis.content.add(item_properties=props, data=file_path)

# Publish item
item.publish(None)


<Item title:"dem_inteporlation" type:Feature Layer Collection owner:zhan6322_UMN>

# upload kriging_difference point accuracy assesment layer to postgis database


In [29]:
#upload using shp2pgsql
# Set the necessary parameters
shp2pgsql_path = r"D:\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"D:\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/dem_BayesianKriging_CV.shp"
srid = 4326
schema_table = "dem_BayesianKriging_difference"
host = "35.193.80.209"
port = 5432
user = "postgres"
db_name = "Lab3"
password = "zyx11457"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table} -c error:double precision"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")




Return code: 0
Error message: could not print result table: Invalid argument
Field measured is an FTDouble with width 19 and precision 11
Field predicted is an FTDouble with width 19 and precision 11
Field error is an FTDouble with width 19 and precision 11
Field stderror is an FTDouble with width 19 and precision 11
Field stdd_error is an FTDouble with width 19 and precision 11
Field normvalue is an FTDouble with width 19 and precision 11
Field crps is an FTDouble with width 19 and precision 11
Field quanval is an FTDouble with width 19 and precision 11
Field source_id is an FTDouble with width 10 and precision 0
Shapefile type: Point
Postgis type: POINT[2]



In [30]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.193.80.209',
                             database = 'Lab3',
                             user = 'postgres',
                             password = 'zyx11457')

# Set the name of the new table in the PostgreSQL database
output_table = "dem_BayesianKriging_difference"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"ALTER TABLE {output_table} ALTER COLUMN error SET DATA TYPE double precision;")
#cursor.execute(f"")

# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

In [31]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.125.39.137:5000/dem_difference"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"E:/coursework/ARCGIS2/LAB3/LAB3"
    output_file_path = os.path.join(output_folder, "dem_inteporlation_difference.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")

Output file path: E:/coursework/ARCGIS2/LAB3/LAB3\dem_inteporlation_difference.geojson
GeoJSON file saved as E:/coursework/ARCGIS2/LAB3/LAB3\dem_inteporlation_difference.geojson


In [32]:
import json

# Read input JSON data from file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation_difference.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type":"FeatureCollection",
    "features":[
        {
            "type":"Feature",
            "geometry":{
                "type":"Point",
                "coordinates":point_data["geometry"]["coordinates"],
            },
            "properties":{"error": point_data["error"]},
        } for point_data in input_data
    ] 
}

# Write output JSON data to file
with open(r'E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation_difference_esri.geojson', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)

In [33]:
import arcgis
gis = arcgis.gis.GIS("home")

# File previously loaded into AGOL (manually)
file_path = r"E:/coursework/ARCGIS2/LAB3/LAB3/dem_inteporlation_difference_esri.geojson"

# Add item 
props = {"title":"dem_inteporlation_difference", "description":"Lab 3", "tags":"GIS 5572", "type":"GeoJson"}
item = gis.content.add(item_properties=props, data=file_path)

# Publish item
item.publish(None)

<Item title:"dem_inteporlation_difference" type:Feature Layer Collection owner:zhan6322_UMN>